<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_0_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,57,hypertrophic scarring
2,84,paraplegic
3,93,shooting
4,97,screwing with my short term memory


In [13]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-0.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,60,cigarette withdrawal
2,108,cramps
3,109,make me drunker
4,133,nightmares


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [34]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.216519,4.631794,0.236190,15:40


In [35]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.519366,4.342258,0.262160,15:45


In [36]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.226686,4.307865,0.264936,15:46


In [37]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.185651,4.281682,0.267828,15:44


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [39]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.091924,6.116920,0.139130,00:24


In [40]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.186285,5.072625,0.182609,00:27


In [41]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.726472,4.724535,0.200000,01:11


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.424741,4.358615,0.234783,01:12
1,5.240178,4.230505,0.234783,01:07


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.003663,4.075118,0.200000,01:10
1,4.828059,3.980322,0.200000,01:11


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.648451,3.844147,0.286957,01:07
1,4.479678,3.805566,0.295652,01:06


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.286695,3.658901,0.304348,01:07
1,4.173800,3.594882,0.295652,01:11


In [46]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.993628,3.659196,0.304348,01:10
1,3.880566,3.591377,0.269565,01:08


In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.691944,3.552020,0.295652,01:11
1,3.646987,3.534150,0.330435,01:10
2,3.486376,3.488874,0.321739,01:08
3,3.331505,3.477767,0.321739,01:09


In [48]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.183872,3.545634,0.313043,01:10
1,3.111490,3.440834,0.321739,01:05
2,2.997842,3.368653,0.365217,01:09
3,2.835718,3.346844,0.373913,01:13


In [49]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.721057,3.388394,0.339130,01:10
1,2.706253,3.550182,0.330435,01:11
2,2.555914,3.520394,0.356522,01:13
3,2.439323,3.503960,0.347826,01:06


In [50]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.368160,3.528719,0.365217,01:06
1,2.332869,3.612694,0.339130,01:09
2,2.206214,3.671067,0.313043,01:06
3,2.079783,3.665236,0.339130,01:07


In [51]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.994993,3.582043,0.321739,01:05
1,1.998253,3.729785,0.347826,01:08
2,1.895784,3.686682,0.347826,01:11
3,1.836128,3.712674,0.347826,01:07


In [52]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.759486,3.678201,0.365217,01:05
1,1.798672,3.836263,0.347826,01:07
2,1.765228,3.879502,0.330435,01:07
3,1.716766,3.877380,0.330435,01:05


In [53]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.558497,3.775506,0.330435,01:09
1,1.649709,3.969483,0.313043,01:10
2,1.572791,3.903813,0.321739,01:10
3,1.516189,3.916584,0.330435,01:13


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [58]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1340, 1597, 947, 1578, 1184, 231, 483, 189, 231, 231, 233, 2147, 233, 233, 249, 249, 1724, 393, 1356, 1519, 1529, 1099, 644, 1942, 749, 749, 2149, 792, 792, 792, 813, 231, 852, 749, 1265, 231, 974, 978, 978, 978, 978, 978, 231, 857, 1007, 2149, 1669, 1767, 1599, 1925, 231, 561, 1136, 1319, 908, 578, 231, 231, 1529, 1787, 1265, 1930, 1295, 1073, 518, 1308, 1340, 231, 1340, 231, 1340, 231, 1973, 1484, 1508, 1701, 1531, 2032, 1597, 1597, 1597, 1597, 1597, 1599, 231, 1942, 231, 231, 1669, 2032, 1915, 359, 1711, 231, 1767, 1767, 1099, 393, 1073, 1915, 1718, 2191, 1767, 1718, 1787, 1787, 2032, 393, 1787, 393, 1099, 1718, 1815, 1815, 1767, 1940, 1769, 1884, 825, 1973, 1930, 1936, 1936, 1940, 1942, 1942, 2002, 2109, 1241, 2018, 1767, 452, 231, 1877, 1884, 1767, 2032, 2147, 974, 852, 1787, 2191, 2191]
[13, 60, 108, 109, 133, 154, 187, 189, 231, 231, 233, 233, 233, 233, 249, 249, 265, 393, 418, 520, 561, 561, 644, 749, 749, 749, 772, 792, 792, 792, 813, 852, 852, 887, 908, 974, 974, 978

In [59]:
correct_num = 0

for i in range(0, 143):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

60
0.4195804195804196
